# New York AirBnB

__Import statements__

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import (
    train_test_split
)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer
)
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

from sklearn.feature_selection import RFECV

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso

from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import StackingRegressor

import eli5

from sklearn.feature_selection import SequentialFeatureSelector

__Importing the dataset__

In [2]:
data = pd.read_csv("../data/raw/airbnb.csv")

/home/utkarshsaboo45/miniconda3/envs/env_airbnb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,48,2019-11-04,0.33,3,322,0,NaN
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,73,1,408,2021-06-29,4.91,1,220,38,NaN
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2016-06-05,0.53,2,365,0,NaN
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,5,2,2021-08-08,0.02,1,91,1,NaN
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,505,2021-10-20,3.70,1,218,31,NaN


In [4]:
# Drop license, id, last_review
# data.drop(columns=["id", "license", "last_review"], inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37713 entries, 0 to 37712
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              37713 non-null  int64  
 1   name                            37700 non-null  object 
 2   host_id                         37713 non-null  int64  
 3   host_name                       37682 non-null  object 
 4   neighbourhood_group             37713 non-null  object 
 5   neighbourhood                   37713 non-null  object 
 6   latitude                        37713 non-null  float64
 7   longitude                       37713 non-null  float64
 8   room_type                       37713 non-null  object 
 9   price                           37713 non-null  int64  
 10  minimum_nights                  37713 non-null  int64  
 11  number_of_reviews               37713 non-null  int64  
 12  last_review                     

__Preprocessing__

In [6]:
data.dropna(subset=["reviews_per_month"], inplace=True)

In [7]:
data["last_review_year"] = pd.DatetimeIndex(data["last_review"]).year
data["last_review_month"] = pd.DatetimeIndex(data["last_review"]).month
data["last_review_date"] = pd.DatetimeIndex(data["last_review"]).day

In [8]:
## Trial but failed

# def find_no_of_listings(data):
#     temp_df = pd.DataFrame(data["host_id"].value_counts()).reset_index()

#     temp_df.rename(columns={
#         "index": "host_id",
#         "host_id": "number_of_listings"
#     }, inplace=True)

#     data = pd.merge(data, temp_df, on="host_id")

#     data["number_of_listings"] = pd.cut(
#         data.number_of_listings,
#         bins=[0, 1, 10, np.inf],
#         labels=["1", "2-10", ">10"]
#     )

#     return data

In [9]:
# # Converting the neighbourhoods occuring less than 86 times to "Others"

# less_frequent_list = X_train["neighbourhood"].value_counts()[86:].index.tolist()
# X_train.loc[X_train.query(
#     "neighbourhood in @less_frequent_list"
# ).index, "neighbourhood"] = "Others"

__Splitting the data__

In [10]:
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=123)
train_data, val_data = train_test_split(train_val_data, test_size=0.25, random_state=123)

In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

__Feature Type Determination__

In [12]:
numeric_features = [
    "latitude",
    "longitude",
    "price",
    "minimum_nights",
    "calculated_host_listings_count",
    "availability_365",
    "last_review_year",
    "last_review_date",
    "id",
    "host_id",
    "number_of_reviews",
    "number_of_reviews_ltm"
]

categorical_features = [
    "neighbourhood_group",
    "room_type",
    "last_review_month"
]

text_features = [
    "name"
]

drop_features = [
    "license",
    "host_name",
    "last_review",
    "neighbourhood"
]

passthrough_features = [

]

target_column = "reviews_per_month"

assert (
    len(numeric_features) +
    len(categorical_features) +
    len(text_features) +
    len(drop_features) +
    len(passthrough_features) +
    len([target_column])
) == len(train_data.columns)

In [13]:
X_train, y_train = train_data.drop(columns=[target_column]), train_data[target_column]
X_val, y_val = val_data.drop(columns=[target_column]), val_data[target_column]
X_train_val, y_train_val = train_val_data.drop(columns=[target_column]), train_val_data[target_column]

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

__Modelling__

In [15]:
function_transformer = FunctionTransformer(
    np.reshape, kw_args={"newshape": -1}
)

pipe_text_feats = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    function_transformer,
    CountVectorizer(stop_words="english", max_features=30)
)

column_transformer = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown="ignore", sparse=False), categorical_features),
    (pipe_text_feats, text_features),
    ("drop", drop_features),
    ("passthrough", passthrough_features)
)

In [16]:
column_transformer.fit(X_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['latitude', 'longitude', 'price',
                                  'minimum_nights',
                                  'calculated_host_listings_count',
                                  'availability_365', 'last_review_year',
                                  'last_review_date', 'id', 'host_id',
                                  'number_of_reviews',
                                  'number_of_reviews_ltm']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['neighbourhoo...
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('functiontransformer',
               

In [17]:
new_columns = (
    numeric_features +
    column_transformer.named_transformers_["onehotencoder"].get_feature_names_out().tolist() +
    column_transformer.named_transformers_["pipeline"].named_steps["countvectorizer"].get_feature_names_out().tolist() +
    passthrough_features
)

In [18]:
pd.DataFrame(column_transformer.transform(X_train), columns=new_columns)

,latitude,longitude,price,minimum_nights,calculated_host_listings_count,availability_365,last_review_year,last_review_date,id,host_id,...,new,nyc,park,private,room,spacious,studio,sunny,village,williamsburg
0,-0.406307,0.111532,-0.088612,0.399872,-0.259772,-0.237283,0.764204,-0.942082,0.240064,-0.705988,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.960255,-0.597747,-0.051700,-0.550828,-0.259772,-0.903438,0.764204,1.002941,1.336932,-0.740110,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.122256,-0.881149,-0.002484,-0.664912,5.811509,-0.266247,0.764204,0.183984,1.530824,0.467355,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.575909,-0.306592,-0.113220,0.399872,-0.155693,-0.135912,0.764204,0.286353,1.274427,2.118255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.913452,1.029390,-0.338795,-0.626884,-0.190386,0.204406,0.764204,-1.453930,-0.186715,0.087669,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17095,1.229243,-0.304653,-0.379808,-0.702940,-0.259772,-0.903438,0.764204,-0.839712,1.434540,-0.406440,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
17096,0.790320,0.746570,-0.404416,0.323816,-0.225079,-0.686214,0.764204,0.695832,1.487830,2.372057,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17097,-0.617706,0.056480,-0.474139,-0.702940,0.052466,1.167433,0.764204,0.491093,0.817329,-0.768836,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
17098,1.412290,-0.017181,-0.310085,-0.626884,-0.259772,-0.816548,-0.314539,0.183984,0.818855,-0.257824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Credits to Varada K.

# def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
#     """
#     Returns mean and std of cross validation

#     Parameters
#     ----------
#     model :
#         scikit-learn model
#     X_train : numpy array or pandas DataFrame
#         X in the training data
#     y_train :
#         y in the training data

#     Returns
#     ----------
#         pandas Series with mean scores from cross_validation
#     """

#     scores = cross_validate(model, X_train, y_train, **kwargs)

#     mean_scores = pd.DataFrame(scores).mean()
#     std_scores = pd.DataFrame(scores).std()
#     out_col = []

#     for i in range(len(mean_scores)):
#         out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

#     return pd.Series(data=out_col, index=mean_scores.index)

In [20]:
def cross_val_scores(model, X_train, y_train, X_val, y_val, return_train_score):

    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    score_dict = {
        "r2_test": model.score(X_val, y_val),
        "mse_test": mean_squared_error(y_val, y_val_pred),
        "mape_test": mean_absolute_percentage_error(y_val, y_val_pred)
    }

    if return_train_score:
        y_train_pred = model.predict(X_train)

        score_dict["r2_train"] = model.score(X_train, y_train)
        score_dict["mse_train"] = mean_squared_error(y_train, y_train_pred)
        score_dict["mape_train"] = mean_absolute_percentage_error(y_train, y_train_pred)

    scores_result = pd.Series(score_dict)

    return model, scores_result

In [21]:
# Adapted from UBC Lecture Notes

def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))

In [22]:
# make a scorer function that we can pass into cross-validation
mape_scorer = make_scorer(mape, greater_is_better=False)

scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2",
    "mape": mape_scorer,
}

__Base Models__

In [23]:
results = {}

In [24]:
models = {
    "Dummy": make_pipeline(
        column_transformer, DummyRegressor()
    ),
    "Ridge": make_pipeline(
        column_transformer, Ridge()
    ),
    "Lasso": make_pipeline(
        column_transformer, Lasso()
    ),
    "Random_Forest_reg": make_pipeline(
        column_transformer, RandomForestRegressor(random_state=123)
    ),
    "XGBoost_reg": make_pipeline(
        column_transformer, XGBRegressor(verbosity=0)
    ),
    "lgbm_reg": make_pipeline(
        column_transformer, LGBMRegressor()
    ),
    "catBoost_reg": make_pipeline(
        column_transformer, CatBoostRegressor(verbose=0)
    )
}

for model_name, model in models.items():
    _, results[model_name] = cross_val_scores(
        model,
        X_train,
        y_train,
        X_val,
        y_val,
        return_train_score=True
    )

In [25]:
pd.DataFrame(results)

,Dummy,Ridge,Lasso,Random_Forest_reg,XGBoost_reg,lgbm_reg,catBoost_reg
r2_test,-0.000010,0.484823,0.407891,0.500858,0.433224,0.504185,0.553416
mse_test,17.962028,9.253537,10.635373,8.965511,10.180341,8.905761,8.021479
mape_test,14.117369,3.872633,8.823251,0.413916,0.915507,0.704105,0.916343
r2_train,0.000000,0.389500,0.307888,0.926435,0.958081,0.795359,0.875625
mse_train,18.968049,11.579991,13.128019,1.395379,0.795113,3.881636,2.359146
mape_train,14.434554,4.038569,9.018054,0.144263,0.603201,0.669306,0.841592


__Feature Selection__

In [26]:
results_cat_boost = {}

n_features_list = [30, 35, 40, 45, 50, 55]

for n_features in n_features_list:
    pipe_forward = make_pipeline(
        column_transformer,
        SequentialFeatureSelector(
            Lasso(),
            direction="forward",
            n_features_to_select=n_features
        ),
        CatBoostRegressor(verbose=0)
    )

    _, results_cat_boost[f"rf_forward_fs_{n_features}"] = cross_val_scores(
        pipe_forward, X_train, y_train, X_val, y_val, return_train_score=True
    )

In [27]:
pd.DataFrame(results_cat_boost)

,rf_forward_fs_30,rf_forward_fs_35,rf_forward_fs_40,rf_forward_fs_45,rf_forward_fs_50,rf_forward_fs_55
r2_test,0.501428,0.509789,0.501390,0.471032,0.516194,0.518276
mse_test,8.955274,8.805092,8.955962,9.501245,8.690054,8.652647
mape_test,1.715783,1.730370,1.738828,1.742144,1.758899,1.776500
r2_train,0.847531,0.845113,0.844549,0.844779,0.843375,0.841289
mse_train,2.892031,2.937907,2.948593,2.944232,2.970871,3.010438
mape_train,1.627030,1.658327,1.673355,1.659658,1.676615,1.693946


In [28]:
rfecv = RFECV(Ridge(), min_features_to_select=40, n_jobs=-1)

pipe_catboost_rfecv = make_pipeline(
    column_transformer, rfecv, CatBoostRegressor(random_state=42, verbose=False)
)

_, results["catboost_rfecv"] = cross_val_scores(
    pipe_catboost_rfecv, X_train, y_train, X_val, y_val, return_train_score=True
)

In [29]:
pd.DataFrame(results)

,Dummy,Ridge,Lasso,Random_Forest_reg,XGBoost_reg,lgbm_reg,catBoost_reg,catboost_rfecv
r2_test,-0.000010,0.484823,0.407891,0.500858,0.433224,0.504185,0.553416,0.547076
mse_test,17.962028,9.253537,10.635373,8.965511,10.180341,8.905761,8.021479,8.135359
mape_test,14.117369,3.872633,8.823251,0.413916,0.915507,0.704105,0.916343,0.963658
r2_train,0.000000,0.389500,0.307888,0.926435,0.958081,0.795359,0.875625,0.868417
mse_train,18.968049,11.579991,13.128019,1.395379,0.795113,3.881636,2.359146,2.495879
mape_train,14.434554,4.038569,9.018054,0.144263,0.603201,0.669306,0.841592,0.897401


In [30]:
pipe_catboost_rfecv.named_steps["rfecv"].n_features_

59

__Polynomial Features Regression__

In [31]:
poly_feats = PolynomialFeatures(degree=2)

pipe_poly = make_pipeline(
    column_transformer, poly_feats, Ridge()
)

_, results["poly_feat"] = cross_val_scores(
    pipe_poly,
    X_train,
    y_train,
    X_val,
    y_val,
    return_train_score=True
)

__Hyperparameter Tuning__

_Ridge_

In [32]:
params = {'ridge__alpha': 10.0 ** np.arange(-2, 6, 1)}

In [33]:
pipe_ridge = make_pipeline(column_transformer, Ridge())

random_search = RandomizedSearchCV(
    pipe_ridge,
    params,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [34]:
random_search.fit(X_train_val, y_train_val)

/home/utkarshsaboo45/miniconda3/envs/env_airbnb/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'price',
                                                                                'minimum_nights',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365',
                                                                                'last_review_year',
                                                     

In [35]:
pd.DataFrame(random_search.cv_results_)[[
        "mean_fit_time",
        "mean_score_time",
        "param_ridge__alpha",
        "mean_train_neg RMSE",
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by='mean_test_r2', ascending=False)

,mean_fit_time,mean_score_time,param_ridge__alpha,mean_train_neg RMSE,std_train_neg RMSE,mean_test_mape,mean_train_mape,mean_test_r2,mean_train_r2
4,0.269386,0.061869,100.0,-3.312132,0.131516,-385.279139,-384.468327,0.409383,0.413215
3,0.266216,0.061504,10.0,-3.311648,0.131473,-393.094137,-391.997237,0.409119,0.413386
2,0.296550,0.062886,1.0,-3.311638,0.131472,-394.137128,-393.012350,0.409064,0.413390
1,0.272395,0.061554,0.1,-3.311638,0.131472,-394.244897,-393.118176,0.409058,0.413390
0,0.270487,0.061324,0.01,-3.311638,0.131472,-394.255704,-393.128797,0.409057,0.413390
5,0.277704,0.064469,1000.0,-3.319000,0.131689,-364.191336,-364.085037,0.408723,0.410781
6,0.265327,0.061851,10000.0,-3.435531,0.133703,-389.226029,-389.504014,0.370300,0.368710
7,0.251320,0.045045,100000.0,-3.954225,0.142924,-907.720645,-907.656488,0.166305,0.163684


In [36]:
pipe_ridge_final = make_pipeline(
    column_transformer,
    Ridge(alpha=1000)
)

pipe_ridge_final.fit(X_train_val, y_train_val)

eli5.explain_weights(pipe_ridge_final.named_steps["ridge"], feature_names=new_columns)

Weight?,Feature
+2.318,number_of_reviews_ltm
+1.479,<BIAS>
+0.499,id
+0.400,last_review_month_10
+0.262,last_review_month_11
+0.241,cozy
+0.190,host_id
+0.135,room_type_Private room
+0.126,number_of_reviews
+0.126,charming


In [37]:
data = {
    "Importance": pipe_ridge_final.named_steps["ridge"].coef_,
}
pd.DataFrame(data=data, index=new_columns,).sort_values(
    by="Importance", ascending=False
)[:10]

,Importance
number_of_reviews_ltm,2.318071
id,0.498539
last_review_month_10,0.400228
last_review_month_11,0.262293
cozy,0.240509
host_id,0.190452
room_type_Private room,0.135238
number_of_reviews,0.125724
charming,0.125625
neighbourhood_group_Bronx,0.096616


_LGBM_

In [38]:
params = {
    'lgbmregressor__n_estimators': [10, 100, 1000],
    'lgbmregressor__max_depth': [5, 10, 15]
}

In [39]:
pipe_lgbm = make_pipeline(column_transformer, LGBMRegressor())

random_search = RandomizedSearchCV(
    pipe_lgbm,
    params,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [40]:
random_search.fit(X_train_val, y_train_val)

/home/utkarshsaboo45/miniconda3/envs/env_airbnb/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'price',
                                                                                'minimum_nights',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365',
                                                                                'last_review_year',
                                                     

In [41]:
pd.DataFrame(random_search.cv_results_)[[
        "mean_fit_time", 
        "mean_score_time", 
        "param_lgbmregressor__n_estimators", 
        "param_lgbmregressor__max_depth",
        "mean_train_neg RMSE", 
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by='mean_test_r2', ascending=False)

,mean_fit_time,mean_score_time,param_lgbmregressor__n_estimators,param_lgbmregressor__max_depth,mean_train_neg RMSE,std_train_neg RMSE,mean_test_mape,mean_train_mape,mean_test_r2,mean_train_r2
1,0.545751,0.083621,100,5,-2.496244,0.075635,-79.441512,-77.674748,0.461116,0.666349
4,0.714639,0.100522,100,10,-2.055569,0.060416,-69.720791,-65.301392,0.441955,0.773767
7,0.737726,0.104741,100,15,-1.954765,0.060678,-72.283469,-67.711531,0.431886,0.795426
0,0.366646,0.072762,10,5,-3.147709,0.087081,-586.921962,-586.435583,0.417316,0.469685
6,0.394178,0.065808,10,15,-3.032879,0.089289,-614.880257,-614.207838,0.417120,0.507745
3,0.392506,0.065683,10,10,-3.034432,0.088136,-613.182519,-612.257177,0.416696,0.507209
2,2.594398,0.563900,1000,5,-1.198019,0.073162,-105.511348,-87.052788,0.402606,0.922889
5,3.844981,0.796645,1000,10,-0.687727,0.061083,-97.150038,-70.598117,0.389419,0.974482
8,3.875973,0.745716,1000,15,-0.603113,0.043318,-88.721957,-61.366587,0.384043,0.980397


_CatBoost_

In [42]:
params = {
    "catboostregressor__iterations": [100, 150, 200],
    "catboostregressor__learning_rate": [0.03, 0.1],
    "catboostregressor__depth": [2, 4, 6, 8],
    "catboostregressor__l2_leaf_reg": [0.2, 0.5, 1, 3]
}

In [43]:
pipe_catboost = make_pipeline(column_transformer, CatBoostRegressor(verbose=False))

random_search = RandomizedSearchCV(
    pipe_catboost,
    params,
    n_jobs=-1,
    n_iter=30,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [44]:
random_search.fit(X_train_val, y_train_val)

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'price',
                                                                                'minimum_nights',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365',
                                                                                'last_review_year',
                                                     

In [45]:
pd.DataFrame(random_search.cv_results_)[
    [
        "mean_fit_time",
        "mean_score_time",
        "param_catboostregressor__iterations",
        "param_catboostregressor__learning_rate",
        "param_catboostregressor__depth",
        "param_catboostregressor__l2_leaf_reg",
        "mean_train_neg RMSE",
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by="mean_test_r2", ascending=False).T

,21,1,19,9,11,18,26,0,4,3,...,24,12,5,10,2,29,16,17,8,25
mean_fit_time,3.827926,6.934258,10.388882,9.099845,7.917222,6.436676,3.816588,5.553577,4.594944,3.604091,...,2.307474,5.685284,3.721268,4.477874,3.461095,2.264113,5.082393,3.173119,4.751504,3.075708
mean_score_time,0.457452,0.291481,0.59173,0.449665,0.686001,0.585055,0.51509,0.517194,0.335876,0.543801,...,0.488815,0.335303,0.514723,0.421966,0.483494,0.244305,0.619909,0.323492,0.564471,0.393133
param_catboostregressor__iterations,100,200,150,150,200,100,100,100,200,100,...,100,200,200,150,150,150,150,200,150,200
param_catboostregressor__learning_rate,0.1,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.1,0.1,0.03,0.1,0.03,0.1,0.03,0.1,0.03,0.1
param_catboostregressor__depth,6,6,8,8,6,8,6,6,4,6,...,2,4,2,2,2,2,2,2,2,2
param_catboostregressor__l2_leaf_reg,1,1,1,0.2,3,0.2,0.2,0.5,3,1,...,1,0.2,0.5,0.5,0.5,0.2,0.2,0.5,3,0.2
mean_train_neg RMSE,-2.37502,-2.547293,-2.432443,-2.327018,-2.689771,-2.493278,-2.693781,-2.725816,-2.842699,-2.753785,...,-2.894005,-2.322776,-2.952143,-2.813035,-3.011286,-2.805745,-2.996758,-2.767293,-3.074582,-2.758047
std_train_neg RMSE,0.043135,0.050235,0.04262,0.048267,0.065531,0.054233,0.066257,0.066231,0.094433,0.067409,...,0.103911,0.054152,0.106682,0.091856,0.11148,0.092671,0.108718,0.086052,0.110936,0.086512
mean_test_mape,-84.968109,-102.745507,-135.556139,-128.663083,-151.597765,-224.828033,-243.656268,-243.91595,-164.791692,-252.262858,...,-139.640137,-91.526469,-134.954335,-133.913598,-250.249985,-138.553941,-243.017383,-138.796933,-282.356861,-143.097967
mean_train_mape,-82.625409,-101.112619,-133.615116,-126.581714,-149.707764,-223.230035,-242.150142,-242.449326,-163.587057,-250.687201,...,-139.208397,-88.992919,-133.755489,-133.234069,-249.0919,-138.071485,-241.932547,-137.740154,-280.671418,-142.208291


__Stacking__

In [46]:
models_selected = {
    "Ridge": make_pipeline(
        column_transformer, PolynomialFeatures(degree=2), Ridge()
    ),
    "Random_Forest_reg": make_pipeline(
        column_transformer, RandomForestRegressor()
    ),
    "lgbm_reg": make_pipeline(
        column_transformer, LGBMRegressor()
    ),
    "catBoost_reg": make_pipeline(
        column_transformer, CatBoostRegressor(verbose=0)
    )
}

In [47]:
stacking_model = StackingRegressor(list(models_selected.items()))

In [48]:
stacking_model, results["Stacking"] = cross_val_scores(
    stacking_model,
    X_train,
    y_train,
    X_val,
    y_val,
    return_train_score=True
)

In [49]:
pd.DataFrame(results)

,Dummy,Ridge,Lasso,Random_Forest_reg,XGBoost_reg,lgbm_reg,catBoost_reg,catboost_rfecv,poly_feat,Stacking
r2_test,-0.000010,0.484823,0.407891,0.500858,0.433224,0.504185,0.553416,0.547076,0.498116,0.568987
mse_test,17.962028,9.253537,10.635373,8.965511,10.180341,8.905761,8.021479,8.135359,9.014759,7.741791
mape_test,14.117369,3.872633,8.823251,0.413916,0.915507,0.704105,0.916343,0.963658,5.197266,1.787546
r2_train,0.000000,0.389500,0.307888,0.926435,0.958081,0.795359,0.875625,0.868417,0.550253,0.815806
mse_train,18.968049,11.579991,13.128019,1.395379,0.795113,3.881636,2.359146,2.495879,8.530831,3.493800
mape_train,14.434554,4.038569,9.018054,0.144263,0.603201,0.669306,0.841592,0.897401,4.635072,1.576291


__Testing on Test Set__

In [50]:
X_test, y_test = test_data.drop(columns=[target_column]), test_data[target_column]

In [51]:
stacking_model.score(X_test, y_test)

0.42244395851143757